In [1]:
import numpy as np
import pandas as pd
 
import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import *
import re

In [2]:
#Загружаем первую часть размеченного корпуса
negative = pd.read_csv("negative.csv", sep=';', header = None)

negative

FileNotFoundError: ignored

In [ ]:
#Загружаем вторую часть размеченного корпуса
positive = pd.read_csv("positive.csv", sep=';', header = None)

positive

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
114906,411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2
114907,411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0
114908,411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13
114909,411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0


In [ ]:
#Выбираем 3 столбец, потому что предстоит работать только с текстом
positive_text = pd.DataFrame(positive.iloc[:, 3])
negative_text = pd.DataFrame(negative.iloc[:, 3])

In [ ]:
#Маркируем позитивные и негативные тексты за 0 и 1 в дополнительную колонку
positive_text['label'] = [1] * positive_text.shape[0]
negative_text['label'] = [0] * negative_text.shape[0]

In [ ]:
#Складываем тексты
labeled_tweets = pd.concat([positive_text, negative_text])

In [ ]:
#Исправим нумерацию корпуса
labeled_tweets.index = range(labeled_tweets.shape[0])

In [ ]:
#Переименуем колонки
labeled_tweets.columns = ['text', 'label']
labeled_tweets

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,0
226830,скучаю так :-( только @taaannyaaa вправляет мо...,0
226831,"Вот и в школу, в говно это идти уже надо(",0
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",0


In [ ]:
#Функция очистки текста от не кириллических символов
def clear_text(text):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    return " ".join(clear_text.split())
    
#Функция очистки текста от наречий, союзов, предлогов и т.д
def clean_stop_words(text, stopwords):
    text = [word for word in text.split() if word not in stopwords]
    return " ".join(text)

In [ ]:
#Загружаем таблицу со стопсловами
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('russian'))
np.array(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


array({'после', 'почти', 'конечно', 'моя', 'того', 'до', 'от', 'без', 'быть', 'ей', 'бы', 'эту', 'эти', 'над', 'если', 'и', 'о', 'зачем', 'или', 'чтоб', 'я', 'наконец', 'уже', 'можно', 'вы', 'из', 'нас', 'был', 'уж', 'им', 'к', 'всегда', 'может', 'про', 'что', 'ним', 'другой', 'сейчас', 'вас', 'хоть', 'ее', 'надо', 'чтобы', 'ты', 'во', 'там', 'ничего', 'как', 'потом', 'ней', 'через', 'будто', 'разве', 'какая', 'но', 'ни', 'а', 'меня', 'впрочем', 'тебя', 'вам', 'чуть', 'том', 'один', 'чего', 'есть', 'со', 'она', 'да', 'его', 'нет', 'в', 'нее', 'всех', 'этом', 'под', 'этот', 'такой', 'же', 'даже', 'совсем', 'кто', 'где', 'два', 'потому', 'перед', 'тот', 'у', 'об', 'между', 'еще', 'себе', 'тем', 'за', 'более', 'всю', 'мне', 'будет', 'тут', 'сам', 'больше', 'хорошо', 'этого', 'вот', 'все', 'нельзя', 'было', 'ли', 'мы', 'были', 'не', 'них', 'него', 'когда', 'ведь', 'раз', 'нибудь', 'ж', 'с', 'себя', 'никогда', 'три', 'свою', 'здесь', 'куда', 'много', 'их', 'ему', 'опять', 'какой', 'то', 'то

In [ ]:
#Делаем очищенные тексты
labeled_tweets['text_clear'] = labeled_tweets['text'].apply(lambda x: clean_stop_words(clear_text(str(x)), stopwords))

In [ ]:
#Меняем тексты на обработанные в корпусе
labeledtweets = labeled_tweets[['text_clear','label']]
labeledtweets.columns = ['text','label']
labeledtweets

,text,label
0,школота поверь самое общество профилирующий пр...,1
1,таки немного похож мальчик равно,1
2,идиотка испугалась,1
3,углу сидит погибает голода ещё порции взяли хо...,1
4,значит страшилка блин посмотрев части создастс...,1
...,...,...
226829,каждый хочет исправлять,0
226830,скучаю вправляет мозги равно скучаю,0
226831,школу говно это идти,0
226832,тауриэль грусти обнял,0


In [ ]:
train, test = train_test_split(labeledtweets,
                        test_size = 0.2,
                        random_state = 12348,
                       )

print(train.shape)
print(test.shape)

(181467, 2)
(45367, 2)


In [ ]:
train

,text,label
162424,учёные заметили кредитных договорах самом низу...,0
160594,капец голова болит заболел походу работу ехать,0
189688,взяла твиттер удалила,0
78348,оно понятно определилась,1
157114,уроки нужно сделать контре подготовиться,0
...,...,...
141717,кажется начинаю остывать это очень ху холодно,0
207453,дорогой дело твое просто очень грустно преврат...,0
36863,председатель правительства рф дмитрий медведев...,1
146555,аххаха баба фрося аххахаха глуши нить,0


In [ ]:
for sample in [train, test]:    
    print(sample[sample['label'] == 1].shape[0] / sample.shape[0])

0.5066265491797407
0.5064253752727753


In [ ]:
count_idf_positive = TfidfVectorizer(ngram_range = (1,1))
count_idf_negative = TfidfVectorizer(ngram_range = (1,1))

tf_idf_positive = count_idf_positive.fit_transform(train.query('label == 1')['text'])
tf_idf_negative = count_idf_negative.fit_transform(train.query('label == 0')['text'])

In [ ]:
positive_importance = pd.DataFrame(
    {'word' : count_idf_positive.get_feature_names(),
     'idf' : count_idf_positive.idf_
    }).sort_values(by = 'idf', ascending = False)

negative_importance = pd.DataFrame(
    {'word' : count_idf_negative.get_feature_names(),
     'idf' : count_idf_negative.idf_
    }).sort_values(by = 'idf', ascending = False)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
positive_importance.query('word not in @negative_importance.word and idf < 15')

,word,idf
104649,ёёёёёёёёёёёёёёёёёёёёёлки,11.735712
92881,ужираться,11.735712
51816,нюансах,11.735712
92883,ужом,11.735712
51812,нээсээ,11.735712
...,...,...
70829,продолжай,9.209983
93370,улыбнуло,8.667659
53367,оглядываются,8.557658
73605,разблокируют,8.537039


In [ ]:
negative_importance.query('word not in @positive_importance.word and idf < 15')

,word,idf
32757,леруша,11.709204
40993,незнакомок,11.709204
41036,неинтересного,11.709204
41037,неинтересное,11.709204
41038,неинтересной,11.709204
...,...,...
68023,скорбим,8.595689
52192,погибших,8.395018
41110,некролог,8.359300
74888,теракт,8.308007


In [ ]:
count_idf_1 = TfidfVectorizer(ngram_range=(1,1))

In [ ]:
tf_idf_base_1 = count_idf_1.fit(labeledtweets['text'])
tf_idf_train_base_1 = count_idf_1.transform(train['text'])
tf_idf_test_base_1 = count_idf_1.transform(test['text'])

In [ ]:
display(tf_idf_test_base_1.shape)
display(tf_idf_train_base_1.shape)

(45367, 170344)

(181467, 170344)

In [ ]:
model_lr_base_1 = LogisticRegression(solver = 'lbfgs',
                                    random_state = 12345,
                                    max_iter= 10000,
                                    n_jobs= -1)

In [ ]:
model_lr_base_1.fit(tf_idf_train_base_1, train['label'])
predict_lr_base_proba = model_lr_base_1.predict_proba(tf_idf_test_base_1)

In [ ]:
weights = pd.DataFrame({'words': count_idf_1.get_feature_names(),
                        'weights': model_lr_base_1.coef_.flatten()})
weights_min = weights.sort_values(by= 'weights')
weights_max = weights.sort_values(by= 'weights', ascending = False)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
weights_min = weights_min[:100]
weights_min['weights'] = weights_min['weights'] * -1
weights_min

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,words,weights
85419,обидно,6.913106
28964,грустно,6.084368
39067,жаль,6.079557
98823,печально,5.833982
132462,скучаю,5.404336
...,...,...
165003,шерлок,2.627264
150793,уехала,2.616249
27141,голова,2.614375
131999,скорбим,2.602911


In [ ]:
weights_max = weights_max[:100]
weights_max

,words,weights
5503,ахаха,4.534308
5502,ахах,4.350795
86118,обожаю,4.257825
20898,вромайдан,4.250786
5539,ахахах,4.214949
...,...,...
123673,рекомендую,2.029362
152732,ураааа,2.025815
67727,майдан,2.022085
134420,снов,2.014393


In [ ]:
vocab = weights.query('weights > 0.25 or weights < -0.25')['words']
vocab

1                ааа
2               аааа
3              ааааа
4             аааааа
5            ааааааа
             ...    
170334     ёёлочкууу
170335        ёёпрст
170339    ёёёлочкааа
170340          ёёёё
170342        ёёёёёё
Name: words, Length: 48832, dtype: object

In [ ]:
count_idf = TfidfVectorizer(vocabulary=vocab,
                            ngram_range=(1,1))

In [ ]:
tf_idf = count_idf.fit_transform(labeledtweets['text'])

tf_idf_train = count_idf.transform(train['text'])
tf_idf_test = count_idf.transform(test['text'])

In [ ]:
display(tf_idf_test.shape)
display(tf_idf_train.shape)

(45367, 48832)

(181467, 48832)

In [ ]:
model_lr_base = LogisticRegression(solver = 'lbfgs',
                                    random_state = 12345,
                                    max_iter= 10000,
                                    n_jobs= -1)

In [ ]:
model_lr_base.fit(tf_idf_train, train['label'])

LogisticRegression(max_iter=10000, n_jobs=-1, random_state=12345)

In [ ]:
predict_lr_base_proba_1 = model_lr_base.predict_proba(tf_idf_test)

In [ ]:
expectedtext = input()

Боже как вы достали серьезно


In [ ]:
print(expectedtext)
print('=======================================')
expectedclean = clean_stop_words((clear_text(expectedtext)), stopwords)
print(expectedclean)

Боже как вы достали серьезно
боже достали серьезно


In [ ]:
df = pd.DataFrame({
    "comment": [expectedtext],
    "text_clear": [expectedclean]})
df

,comment,text_clear
0,Боже как вы достали серьезно,боже достали серьезно


In [ ]:
source = df.sample(n=1)
text_clear = source['text_clear'].values[0]
text = source['comment'].values[0]

print(text)

tf_idf_text = count_idf.transform([text_clear])

toxic_proba = model_lr_base.predict_proba(tf_idf_text)

print('Вероятность негатива: ', toxic_proba[:, 0])
print()

Боже как вы достали серьезно
Вероятность негатива:  [0.86083289]



In [ ]:
confusion_matrix(test['label'],
                  (predict_lr_base_proba[:, 1] > 0.5).astype('float'),
                  normalize='true')

array([[0.69087174, 0.30912826],
       [0.22611534, 0.77388466]])

In [ ]:
accuracy = (0.69087174 + 0.77388466) / 2 * 100
print('Точность Логистической регрессии:', accuracy, '%')

Точность Логистической регрессии: 73.23782 %
